In [ ]:
         #ANÁLISIS DE DATOS DE E-COMMERCE - PROYECTO 

# Importamos las bibliotecas necesarias
import pandas as pd      
import numpy as np       
import matplotlib.pyplot as plt  
import seaborn as sns    
import os                
from datetime import datetime  
from matplotlib.ticker import FuncFormatter  

# Cargamos los 
clientes = pd.read_csv('customers_dataset.csv')
ubicaciones = pd.read_csv('geolocation_dataset.csv') 
pagos = pd.read_csv('order_payments_dataset.csv')
productos = pd.read_csv('products_dataset.csv')
vendedores = pd.read_csv('sellers_dataset.csv')
ordenes = pd.read_csv('orders_dataset.csv')
items_ordenes = pd.read_csv('order_items_dataset.csv')
reviews = pd.read_csv('order_reviews_dataset.csv')
traduccion_categorias = pd.read_csv('product_category_name_translation.csv')

# LIMPIEZA Y PREPARACIÓN DE DATOS (DATA WRANGLING)

print("Iniciando proceso de limpieza y preparación de datos...")

# 1. Verificación de datos faltantes
print("\n--- Verificando valores faltantes ---")
print("Clientes - valores faltantes:", clientes.isnull().sum().sum())
print("Ubicaciones - valores faltantes:", ubicaciones.isnull().sum().sum())
print("Pagos - valores faltantes:", pagos.isnull().sum().sum())
print("Productos - valores faltantes:", productos.isnull().sum().sum())
print("Vendedores - valores faltantes:", vendedores.isnull().sum().sum())
print("Ordenes - valores faltantes:", ordenes.isnull().sum().sum())
print("Items ordenes - valores faltantes:", items_ordenes.isnull().sum().sum())
print("Reviews - valores faltantes:", reviews.isnull().sum().sum())

# 2. Limpieza de datos - Eliminamos duplicados
clientes = clientes.drop_duplicates()
ubicaciones = ubicaciones.drop_duplicates()
pagos = pagos.drop_duplicates()
productos = productos.drop_duplicates()
vendedores = vendedores.drop_duplicates()
ordenes = ordenes.drop_duplicates()
items_ordenes = items_ordenes.drop_duplicates()
reviews = reviews.drop_duplicates()

# 3. Convertimos fechas a formato datetime
# Convertimos las fechas en la tabla de órdenes
ordenes['order_purchase_timestamp'] = pd.to_datetime(ordenes['order_purchase_timestamp'])
ordenes['order_approved_at'] = pd.to_datetime(ordenes['order_approved_at'])
ordenes['order_delivered_carrier_date'] = pd.to_datetime(ordenes['order_delivered_carrier_date'])
ordenes['order_delivered_customer_date'] = pd.to_datetime(ordenes['order_delivered_customer_date'])
ordenes['order_estimated_delivery_date'] = pd.to_datetime(ordenes['order_estimated_delivery_date'])

# Convertimos las fechas en la tabla de reviews
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])

# 4. Identificamos valores extremos en precios
precio_promedio = items_ordenes['price'].mean()
precio_desviacion = items_ordenes['price'].std()
precio_minimo = items_ordenes['price'].min()
precio_maximo = items_ordenes['price'].max()

print("\n--- Análisis de precios ---")
print("Precio promedio:", precio_promedio)
print("Precio mínimo:", precio_minimo)
print("Precio máximo:", precio_maximo)

# 5. Traducción de categorías de productos
productos = productos.merge(traduccion_categorias, on='product_category_name', how='left')

# 6. Creamos columnas nuevas útiles
# Añadimos mes y año a las órdenes
ordenes['mes_compra'] = ordenes['order_purchase_timestamp'].dt.month
ordenes['año_compra'] = ordenes['order_purchase_timestamp'].dt.year

# Calculamos tiempo de entrega en días
ordenes['tiempo_entrega'] = (ordenes['order_delivered_customer_date'] - ordenes['order_purchase_timestamp']).dt.days

# 7. Mostramos información después de la limpieza
print("\n--- Resumen de datos después de la limpieza ---")
print("Clientes:", clientes.shape[0], "filas")
print("Ubicaciones:", ubicaciones.shape[0], "filas")
print("Pagos:", pagos.shape[0], "filas")
print("Productos:", productos.shape[0], "filas")
print("Vendedores:", vendedores.shape[0], "filas")
print("Ordenes:", ordenes.shape[0], "filas")
print("Items ordenes:", items_ordenes.shape[0], "filas")
print("Reviews:", reviews.shape[0], "filas")

print("\nProceso de limpieza y preparación de datos completado.")

# RFM ANALYSIS

# Calculamos la fecha más reciente en el dataset
fecha_maxima = ordenes['order_purchase_timestamp'].max()

# Unimos las tablas necesarias para el análisis RFM
df_rfm = ordenes.merge(items_ordenes, on='order_id')

# Calculamos los componentes del RFM por cliente
rfm_data = df_rfm.groupby('customer_id').agg({
    'order_purchase_timestamp': lambda x: (fecha_maxima - x.max()).days,  # Recency
    'order_id': 'nunique',  # Frequency
    'price': 'sum'  # Monetary
}).rename(columns={
    'order_purchase_timestamp': 'recency',
    'order_id': 'frequency',
    'price': 'monetary'
})

# Verificamos los resultados del cálculo RFM
print("Análisis RFM - Primeras filas:")
print(rfm_data.head())

# Estadísticas descriptivas del RFM
print("\nEstadísticas descriptivas del RFM:")
print(rfm_data.describe())

# Segmentación de clientes basada en RFM
# Dividimos cada métrica en 5 segmentos (quintiles)
quintiles = rfm_data.rank(pct=True).applymap(lambda x: int(x * 5) + 1)

# Invertimos la recencia (menor recencia = mejor puntuación)
quintiles['recency'] = 6 - quintiles['recency']

# Creamos una puntuación RFM combinada
rfm_data['rfm_score'] = quintiles['recency'] * 100 + quintiles['frequency'] * 10 + quintiles['monetary']

# Clasificamos a los clientes según su puntuación
def clasificar_cliente(row):
    if row['rfm_score'] >= 432:
        return 'Campeones'
    elif row['rfm_score'] >= 332:
        return 'Leales'
    elif row['rfm_score'] >= 232:
        return 'Potenciales'
    elif row['rfm_score'] >= 132:
        return 'En riesgo'
    else:
        return 'Necesitan atención'

rfm_data['segmento'] = rfm_data.apply(clasificar_cliente, axis=1)

# Visualizamos la distribución de segmentos
sns.set_style("whitegrid") # 1. Seaborn Style
plt.figure(figsize=(12, 7)) # Adjusted figure size for better label display
ax = sns.countplot(x='segmento', data=rfm_data, order=rfm_data['segmento'].value_counts().index, palette="viridis", edgecolor='black') # 2. Color Palette, 5. Bar Edge Color
plt.title('Distribución de Clientes por Segmento RFM', fontsize=16) # 4. Font Size, 6. Clarity
plt.xlabel('Segmento', fontsize=14) # 4. Font Size
plt.ylabel('Número de Clientes', fontsize=14) # 4. Font Size
plt.xticks(rotation=45, fontsize=12) # 4. Font Size
# 3. Data Labels
for p in ax.patches:
    ax.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                textcoords='offset points')
plt.tight_layout()
plt.savefig('segmentos_rfm.png') # 7. Save
plt.close() # 7. Close

print("\nDistribución de clientes por segmento:")
print(rfm_data['segmento'].value_counts())
# Análisis más profundo de los segmentos

# Estadísticas descriptivas por segmento
print("\nEstadísticas por segmento:")
segment_stats = rfm_data.groupby('segmento').agg({
    'recency': ['mean', 'min', 'max'],
    'frequency': ['mean', 'min', 'max'],
    'monetary': ['mean', 'min', 'max']
})
print(segment_stats)

# Visualización de las métricas RFM por segmento
fig, axes = plt.subplots(3, 1, figsize=(15, 12)) # Adjusted figsize
fig.suptitle('Análisis Detallado de Métricas RFM por Segmento', fontsize=18) # 3. Overall Figure Title, 5. Font Sizes
order_segmentos = ['Campeones', 'Leales', 'Potenciales', 'En riesgo', 'Necesitan atención']

# Recencia promedio por segmento
ax1 = axes[0]
sns.barplot(x='segmento', y='recency', data=rfm_data, estimator=np.mean, 
            order=order_segmentos, ax=ax1, palette="viridis", edgecolor='black') # 2. Color Palette, 6. Bar Edge Color
ax1.set_title('Recencia Promedio por Segmento', fontsize=14) # 5. Font Sizes
ax1.set_ylabel('Recencia Promedio (Días)', fontsize=12) # 7. Y-axis Labels, 5. Font Sizes
ax1.set_xlabel('') # 8. X-axis Labels (remove)
ax1.tick_params(axis='x', rotation=45, labelsize=10) # 5. Font Sizes
for p in ax1.patches: # 4. Data Labels
    ax1.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=9, color='black', xytext=(0, 5),
                textcoords='offset points')

# Frecuencia promedio por segmento
ax2 = axes[1]
sns.barplot(x='segmento', y='frequency', data=rfm_data, estimator=np.mean,
            order=order_segmentos, ax=ax2, palette="viridis", edgecolor='black') # 2. Color Palette, 6. Bar Edge Color
ax2.set_title('Frecuencia Promedio por Segmento', fontsize=14) # 5. Font Sizes
ax2.set_ylabel('Frecuencia Promedio (Pedidos)', fontsize=12) # 7. Y-axis Labels, 5. Font Sizes
ax2.set_xlabel('') # 8. X-axis Labels (remove)
ax2.tick_params(axis='x', rotation=45, labelsize=10) # 5. Font Sizes
for p in ax2.patches: # 4. Data Labels
    ax2.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=9, color='black', xytext=(0, 5),
                textcoords='offset points')

# Valor monetario promedio por segmento
ax3 = axes[2]
sns.barplot(x='segmento', y='monetary', data=rfm_data, estimator=np.mean,
            order=order_segmentos, ax=ax3, palette="viridis", edgecolor='black') # 2. Color Palette, 6. Bar Edge Color
ax3.set_title('Valor Monetario Promedio por Segmento', fontsize=14) # 5. Font Sizes
ax3.set_ylabel('Valor Monetario Promedio ($)', fontsize=12) # 7. Y-axis Labels, 5. Font Sizes
ax3.set_xlabel('') # 8. X-axis Labels (remove)
ax3.tick_params(axis='x', rotation=45, labelsize=10) # 5. Font Sizes
for p in ax3.patches: # 4. Data Labels
    ax3.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=9, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.tight_layout(rect=[0, 0, 1, 0.96]) # 9. Layout (adjust for suptitle)
plt.savefig('metricas_por_segmento.png') # 10. Save
plt.close() # 10. Close


In [ ]:
# HEATMAP DE MÉTRICAS RFM PROMEDIO POR SEGMENTO

# 1. Preparamos los datos para el heatmap
segment_metrics_avg = rfm_data.groupby('segmento').agg(
    recency_avg=('recency', 'mean'),
    frequency_avg=('frequency', 'mean'),
    monetary_avg=('monetary', 'mean')
)
segment_order = ['Campeones', 'Leales', 'Potenciales', 'En riesgo', 'Necesitan atención']
segment_metrics_avg = segment_metrics_avg.reindex(segment_order)

# Renombrar columnas para el heatmap
segment_metrics_avg.columns = ['Recencia Promedio', 'Frecuencia Promedio', 'Valor Monetario Promedio']

# 2. Creamos el heatmap
plt.figure(figsize=(12, 7)) # Adjusted figsize
sns.heatmap(segment_metrics_avg, annot=True, fmt=".1f", cmap="viridis", linewidths=.5, cbar_kws={'label': 'Escala de Valores'})
plt.title('Heatmap de Métricas RFM Promedio por Segmento', fontsize=16)
plt.ylabel('Segmento de Cliente', fontsize=12)
plt.xlabel('Métricas RFM Promedio', fontsize=12)
plt.xticks(rotation=45, ha="right") # Rotate x-axis labels for better fit
plt.yticks(fontsize=10)
plt.tight_layout()
plt.savefig('rfm_heatmap_segmentos.png')
plt.close()

print("\nHeatmap de métricas RFM por segmento guardado como 'rfm_heatmap_segmentos.png'")


In [ ]:
# SCATTER PLOT DE RECENCIA VS. FRECUENCIA (COLOR/TAMAÑO POR VALOR MONETARIO)

# 1. Preparamos los datos (usamos rfm_data directamente)
#    Considerar muestreo si el dataset es muy grande y el plot es lento o muy denso:
#    plot_data = rfm_data.sample(n=20000, random_state=42) if len(rfm_data) > 20000 else rfm_data
plot_data = rfm_data # Usar rfm_data directamente primero

# 2. Creamos el scatter plot
plt.figure(figsize=(14, 9)) # Adjusted figsize
scatter_plot = sns.scatterplot(
    x='recency',
    y='frequency',
    hue='monetary',
    size='monetary',  # Optional: use monetary value for size as well
    sizes=(20, 200),  # Range of point sizes
    palette='viridis',
    data=plot_data,
    alpha=0.6, # Adjust alpha for point transparency if there's overplotting
    edgecolor='black', # Add edgecolor to points
    linewidth=0.5
)

plt.title('Relación Recencia vs. Frecuencia (Color/Tamaño por Valor Monetario)', fontsize=16)
plt.xlabel('Recencia (Días desde última compra)', fontsize=12)
plt.ylabel('Frecuencia (Número de Pedidos)', fontsize=12)

# Mejorar la leyenda
handles, labels = scatter_plot.get_legend_handles_labels()
# Podría ser necesario ajustar la leyenda si es muy larga o compleja,
# por ahora, se usa la leyenda por defecto que genera seaborn.
# Ejemplo: plt.legend(title='Valor Monetario', loc='upper right', bbox_to_anchor=(1.25, 1))

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.savefig('rfm_scatter_recency_frequency.png')
plt.close()

print("\nScatter plot de Recencia vs. Frecuencia guardado como 'rfm_scatter_recency_frequency.png'")


In [1]:

# Recomendaciones de marketing basadas en segmentos
print("\nRecomendaciones de marketing por segmento:")
print("1. Campeones: Programas de fidelización, convertirlos en embajadores de marca")
print("2. Leales: Venta cruzada, programas de referidos")
print("3. Potenciales: Ofertas personalizadas, incentivos para aumentar frecuencia")
print("4. En riesgo: Campañas de reactivación, descuentos especiales")
print("5. Necesitan atención: Recordatorios, ofertas de bienvenida nuevamente")

# Exportamos los resultados para uso en campañas de marketing
rfm_data.to_csv('segmentacion_clientes_rfm.csv', index=True)
print("\nResultados exportados a 'segmentacion_clientes_rfm.csv'")


   
    


Iniciando proceso de limpieza y preparación de datos...

--- Verificando valores faltantes ---
Clientes - valores faltantes: 0
Ubicaciones - valores faltantes: 0
Pagos - valores faltantes: 0
Productos - valores faltantes: 2448
Vendedores - valores faltantes: 0
Ordenes - valores faltantes: 4908
Items ordenes - valores faltantes: 0
Reviews - valores faltantes: 145903

--- Análisis de precios ---
Precio promedio: 120.65373901464716
Precio mínimo: 0.85
Precio máximo: 6735.0

--- Resumen de datos después de la limpieza ---
Clientes: 99441 filas
Ubicaciones: 738332 filas
Pagos: 103886 filas
Productos: 32951 filas
Vendedores: 3095 filas
Ordenes: 99441 filas
Items ordenes: 112650 filas
Reviews: 99224 filas

Proceso de limpieza y preparación de datos completado.
Análisis RFM - Primeras filas:
                                  recency  frequency  monetary
customer_id                                                   
00012a2ce6f8dcda20d059ce98491703      337          1     89.80
000161a058600d59